In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Sequential
tf.keras.datasets.cifar10.load_data()

import numpy as np
import matplotlib.pyplot as plt

(x_train, _), (x_test, _) = keras.datasets.cifar10.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 32, 32, 3))
x_test = np.reshape(x_test, (len(x_test), 32, 32, 3))

# #Displaying images
# plt.figure(figsize=(20, 2))
# for i in range(1,10):
#     ax = plt.subplot(1, 10, i)
#     plt.imshow(x_test[i].reshape(32,32,3))
# plt.show()

#Encoder
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32,32,3)))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))    

#decoder
model2 = Sequential()
model2.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32,32,3)))
model2.add(MaxPooling2D((2, 2), padding='same'))
# model2.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
# model2.add(MaxPooling2D((2, 2), padding='same'))
model2.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model2.add(MaxPooling2D((2, 2), padding='same'))    
model2.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model2.add(UpSampling2D((2, 2)))
# model2.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
# model2.add(UpSampling2D((2, 2)))
model2.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model2.add(Conv2D(3, (3, 3), activation='relu', padding='same'))
model2.add(Conv2D(3, (1, 1), activation='sigmoid', padding='same'))
model2.add(UpSampling2D((2, 2)))
model2.add(Conv2D(3, (3, 3), activation='relu', padding='same'))




model.compile(optimizer='adam', loss='mean_squared_error')
model2.compile(optimizer='adam', loss='mean_squared_error')

model.summary()
model2.summary()
compressed_img_train = model.predict(x_train)
compressed_img_test = model.predict(x_test)

model2.fit(x_train, x_train, epochs=5, batch_size=256, shuffle=True,
         validation_data=(x_test,x_test))

model2.evaluate(x_test,x_test )

reconstructed_img = model2.predict(x_test)

# encoder output
compressed_img = model.predict(x_test)
plt.figure(figsize=(40, 4))
for i in range(10):
    # display original
    ax = plt.subplot(3, 20, i + 1)
    plt.imshow(x_test[i].reshape(32,32,3), cmap="binary")
    
    # display compressed image
    ax = plt.subplot(3, 20, 40 +i+ 1)
    plt.imshow(compressed_img[i].reshape((4, 4 * 8)).T, cmap="binary")
plt.show()

##decoder output
plt.figure(figsize=(40, 4))
for i in range(10):
    # display original
    ax = plt.subplot(3, 20, i + 1)
    plt.imshow( x_test[i].reshape(32,32,3) )
  
    # display reconstructed image
    ax = plt.subplot(3, 20, 40 +i+ 1)
    plt.imshow(reconstructed_img[i].reshape(32,32,3))

plt.show()